# Model input ERP format preparation 

## Imports

In [2]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [3]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [4]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,eeg_path,path_epoch,epoch_file
0,101a,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,101a_epo.fif
1,101b,101,b,m,23,23;16,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,101b_epo.fif
2,102a,102,a,f,20,20;27,Nee,Control,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,102a_epo.fif
3,102b,102,b,f,23,23;16,Nee,Control,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,102b_epo.fif
4,103a,103,a,f,20,20;23,m,At risk,F:\Stage\ePODIUM\Data\ePodium (Projectfolder)\...,F:\Stage\ePODIUM\ePodium\dyslexiaprediction_na...,103a_epo.fif


In [5]:
dataframe = metadata
epochs = []
for index, file in dataframe.iterrows():
    print(f"Checking out file: {file['epoch_file']}")
    path = os.path.join(file['path_epoch'], file['epoch_file'])
    epoch = mne.read_epochs(path, preload=False)
    epochs.append(epoch)

Checking out file: 101a_epo.fif
Reading F:\Stage\ePODIUM\ePodium\dyslexiaprediction_nadine\epochs\101a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 101b_epo.fif
Reading F:\Stage\ePODIUM\ePodium\dyslexiaprediction_nadine\epochs\101b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 102a_epo.fif
Reading F:\Stage\ePODIUM\ePodium\dyslexiaprediction_nadine\epochs\102a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 p

In [8]:
# create evoked responses for all events per participant
event_dictionary = epod_helper.event_dictionary
evoked = hf.evoked_responses(epochs, event_dictionary)

In [11]:
# std_evoked = epochs[i][2,5,8,11].average()
# dev_evoked = epochs[i][3,6,9,12].average()

# select evoked responses for specific events
i = 0
std_evoked = epochs[i][2].average()
dev_evoked = epochs[i][3].average()

In [12]:
# substract deviant from standard in order to calculate the mismatch response
evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])

In [27]:
# select the first channel
chnames_list = evoked_diff.info['ch_names']

chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=['Fp1'])

In [36]:
# get the evokeds from the first channel
roi_dict = dict(left_ROI=chnames) # unfortunately combine_channels only takes a dictionary as input
roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
test = roi_evoked.to_data_frame()

<ipython-input-36-84f8a509f02b>:3: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')


In [49]:
mean = test['left_ROI'].mean()
mean

3.981499047325405

In [55]:
# now combine everything in a loop 
standard_events = [2,5,8,11]
deviant_events = [3,6,9,12]

df = pd.DataFrame(columns=["participant", "channel", "mean"])
for i in range(len(evoked)):
    for j in standard_events: # hard coded for the deviant
        # print(j+1)
        std_evoked = epochs[i][j].average()
        dev_evoked = epochs[i][j+1].average()
        
        evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
        chnames_list = evoked_diff.info['ch_names']
        for channel in chnames_list: 
            chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            roi_dict = dict(left_ROI=chnames) # unfortunately combine_channels only takes a dictionary as input
            roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            timeseries = roi_evoked.to_data_frame()
            timeseries_avg = timeseries['left_ROI'].mean()
            df = df.append({'participant': i+1, 'channel': channel, 'mean' :  timeseries_avg}, ignore_index=True) 

df

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

...

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-55-58bdb4ccb025>:17: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000206AEDB8C10>, axis=0)".
  roi_e

,participant,channel,mean
0,1,Fp1,3.981499
1,1,AF3,11.639081
2,1,F7,-18.811622
3,1,F3,18.343454
4,1,FC1,10.023632
...,...,...,...
635,5,F8,-59.793993
636,5,AF4,-69.507558
637,5,Fp2,-66.936390
638,5,Fz,-66.624167
